In [1]:
## COMBINE P-VALUES ##

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

def fisher_method_log10(pvalues):
    df = 2 * len(pvalues)
    return -np.log10(1 - chi2.cdf(-2 * sum(map(np.log, pvalues)), df))

In [21]:

relate_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele"

kasper_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5"

print("First 5 rows of relate file:")
file1 = pd.read_csv(relate_LWK, sep=' ')
print(file1.head())

# Read and print first 5 rows of HDF5 file
print("\nFirst 5 rows of kasper file:")
file2 = pd.read_hdf(kasper_LWK)
print(file2.head())


First 5 rows of relate file:
       pos rs_id  3571428.500000  357142.937500  257030.656250  184981.281250  \
0  2781642     .               1              1              1              1   
1  2781865     .               1              1              1              1   
2  2781927     .               1              1              1              1   
3  2782572     .               1              1              1              1   
4  2783658     .               1              1              1              1   

   133128.375000  95810.585938  68953.507812  49624.847656  ...  257.030640  \
0              1             1             1           1.0  ...   -0.118409   
1              1             1             1           1.0  ...   -0.929843   
2              1             1             1           1.0  ...   -1.058240   
3              1             1             1           1.0  ...   -0.186075   
4              1             1             1           1.0  ...    1.000000   

   184.98

In [22]:
# Get the number of rows and columns for each DataFrame
relate_rows, relate_columns = file1.shape
kasper_rows, kasper_columns = file2.shape

# Print the number of rows and columns for each DataFrame
print("Number of rows and columns in relate DataFrame:")
print("Rows:", relate_rows)
print("Columns:", relate_columns)

print("\nNumber of rows and columns in kasper DataFrame:")
print("Rows:", kasper_rows)
print("Columns:", kasper_columns)


Number of rows and columns in relate DataFrame:
Rows: 83674
Columns: 35

Number of rows and columns in kasper DataFrame:
Rows: 83273
Columns: 13


In [3]:
# Open the files for reading and the new file for writing
with open("/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele", "r") as file1, open("/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5", "r") as file2, open("combined_pvalues_LWK.csv", "w") as combined_file:
    # Skip the header lines in both files
    next(file1) # file1 = Relate
    next(file2) # file2 = Kasper

    # Write the header line in the new file
    combined_file.write("pos,combined_pvalue\n")

    # Iterate through the rows of both files simultaneously until the end of either file is reached
    while True:
        # Read a line from each file
        line1 = file1.readline()
        line2 = file2.readline()

        # Check if either file has reached its end
        if not line1 or not line2:
            break

        # Parse the genomic position and p-value from each line
        pos1, p_value1 = map(float, line1.strip().split(","))
        pos2, p_value2 = map(float, line2.strip().split(","))

        # Check if the genomic positions match
        if pos1 != pos2:
            raise ValueError("Genomic positions do not match between files.")

        # Combine p-values using Fisher's method
        combined_pvalue = fisher_method_log10([p_value1, p_value2])

        # Write combined genomic position and p-value to the new file
        combined_file.write(f"{pos1},{combined_pvalue}\n")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [9]:
import pandas as pd

def fisher_method_log10(pvalues):
    df = 2 * len(pvalues)
    log_pvalues = np.log(np.maximum(pvalues, np.finfo(float).eps))  # Add a small constant to avoid zero or negative p-values
    return -np.log10(1 - chi2.cdf(-2 * sum(log_pvalues), df))


# File paths
relate_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele"
kasper_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5"

# Read files into DataFrames
file1 = pd.read_csv(relate_LWK, sep=' ')
file2 = pd.read_hdf(kasper_LWK, dtype={'pos': int})  # Specify 'pos' column as integer

print(file1.head())
print(file2.head())

# Open the combined file for writing
with open("combined_pvalues_LWK.csv", "w") as combined_file:
    # Write the header line in the new file
    combined_file.write("pos,combined_pvalue\n")

    # Iterate through the rows of both DataFrames simultaneously
  #  for index, row1 in file1.iterrows():
   #     # Find corresponding row in file2 based on 'pos' column
    #    row2 = file2[file2['pos'] == row1['pos']]
#
 #       # Check if corresponding row exists in file2
  #      if not row2.empty:
   ##         # Combine p-values using Fisher's method
     #       combined_pvalue = fisher_method_log10([row1['when_mutation_has_freq2'], row2['p']])

            # Write combined genomic position and p-value to the new file
      #      combined_file.write(f"{row1['pos']},{combined_pvalue}\n")
       # else:
        #    print(f"No corresponding row found in file2 for position {row1['pos']}")


       pos rs_id  3571428.500000  357142.937500  257030.656250  184981.281250  \
0  2781642     .               1              1              1              1   
1  2781865     .               1              1              1              1   
2  2781927     .               1              1              1              1   
3  2782572     .               1              1              1              1   
4  2783658     .               1              1              1              1   

   133128.375000  95810.585938  68953.507812  49624.847656  ...  257.030640  \
0              1             1             1           1.0  ...   -0.118409   
1              1             1             1           1.0  ...   -0.929843   
2              1             1             1           1.0  ...   -1.058240   
3              1             1             1           1.0  ...   -0.186075   
4              1             1             1           1.0  ...    1.000000   

   184.981262  133.128357  95.810577  

In [10]:
import pandas as pd

# File paths
relate_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele"
kasper_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5"

# Read files into DataFrames
file1 = pd.read_csv(relate_LWK, sep=' ')
file2 = pd.read_hdf(kasper_LWK)

# Convert 'pos' column to integers in the second DataFrame
file2['pos'] = file2['pos'].astype(int)

print(file1.head())
print(file2.head())

       pos rs_id  3571428.500000  357142.937500  257030.656250  184981.281250  \
0  2781642     .               1              1              1              1   
1  2781865     .               1              1              1              1   
2  2781927     .               1              1              1              1   
3  2782572     .               1              1              1              1   
4  2783658     .               1              1              1              1   

   133128.375000  95810.585938  68953.507812  49624.847656  ...  257.030640  \
0              1             1             1           1.0  ...   -0.118409   
1              1             1             1           1.0  ...   -0.929843   
2              1             1             1           1.0  ...   -1.058240   
3              1             1             1           1.0  ...   -0.186075   
4              1             1             1           1.0  ...    1.000000   

   184.981262  133.128357  95.810577  

In [19]:
import pandas as pd
import numpy as np
from scipy.stats import chi2


def fisher_method_log10(pvalues):
    df = 2 * len(pvalues)
    log_pvalues = np.log(np.maximum(pvalues, np.finfo(float).eps))  # Add a small constant to avoid zero or negative p-values
    sum_log_pvalues = -2 * sum(log_pvalues)
    return -np.log10(1 - chi2.cdf(sum_log_pvalues, df))




# File paths
relate_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele"
kasper_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5"

# Read files into DataFrames
file1 = pd.read_csv(relate_LWK, sep=' ')
file2 = pd.read_hdf(kasper_LWK)

# Convert 'pos' column to integers in the second DataFrame
file2['pos'] = file2['pos'].astype(int)

# Merge the two DataFrames on the 'pos' column
merged_df = pd.merge(file1, file2, on='pos', how='inner')

print(merged_df.head())

       pos rs_id  3571428.500000  357142.937500  257030.656250  184981.281250  \
0  2781642     .               1              1              1              1   
1  2781865     .               1              1              1              1   
2  2781927     .               1              1              1              1   
3  2782572     .               1              1              1              1   
4  2783658     .               1              1              1              1   

   133128.375000  95810.585938  68953.507812  49624.847656  ...         p  \
0              1             1             1           1.0  ...  1.000000   
1              1             1             1           1.0  ...  0.552704   
2              1             1             1           1.0  ...  0.973672   
3              1             1             1           1.0  ...  0.357742   
4              1             1             1           1.0  ...  1.000000   

   stat  nr_coal           t1  t2  tree_left  tree

In [21]:
# File paths
relate_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele"
kasper_LWK = "/home/ari/ari-intern/people/ari/ariadna-intern/results/1000g_ppl_phased_haplotypes_LWK_runstats.h5"

# Read files into DataFrames
file1 = pd.read_csv(relate_LWK, sep=' ')
file2 = pd.read_hdf(kasper_LWK)

# Convert 'pos' column to integers in the second DataFrame
file2['pos'] = file2['pos'].astype(int)

# Merge the two DataFrames on the 'pos' column
merged_df = pd.merge(file1[['pos', 'when_mutation_has_freq2']], file2[['pos', 'p']], on='pos', how='inner')

# Apply -np.log10() function to the 'p' column
merged_df['p'] = -np.log10(merged_df['p'])

# Output the merged DataFrame
print(merged_df.head())


       pos  when_mutation_has_freq2         p
0  2781642                -0.082692 -0.000000
1  2781865                -0.489434  0.257507
2  2781927                -1.083170  0.011587
3  2782572                -0.145357  0.446430
4  2783658                -0.610668 -0.000000
